In [1]:
import gym

env = gym.make("CartPole-v1")
obs = env.reset()
obs
img = env.render(mode="rgb_array")
img.shape

AttributeError: module 'time' has no attribute 'clock'

In [ ]:
import matplotlib.pyplot as plt

# def plot_environment(env, figsize=(5,4)):
    # plt.figure(figsize=figsize)
    # img = env.render(mode="rgb_array")
    # plt.imshow(img)
    # plt.axis("off")
    # return img

In [ ]:
action = 1
obs, reward, done, info = env.step(action)
obs


array([ 0.0474581 ,  0.18644694,  0.02564116, -0.23659075])

In [ ]:
def basic_policy(obs):
    angle = obs[2]
    return 0 if angle < 0 else 1

totals = []
for episode in range(500):
    episode_rewards = 0
    obs = env.reset()
    for step in range(200):
        action = basic_policy(obs)
        obs, reward, done, info = env.step(action)
        episode_rewards += reward
        if done:
            break
    totals.append(episode_rewards)

In [ ]:
import numpy as np 
np.mean(totals), np.std(totals), np.min(totals), np.max(totals)

42.248